<h1>F20BC Coursework</h1>
<p style="font-size:15px;">This notebook implements and analyses the Biologically-Inspired Computation coursework, focusing on Artificial Neural Networks (ANNs) and Particle Swarm Optimization (PSO). It includes a multi-layer ANN architecture trained using PSO to solve a regression task predicting concrete compressive strength. Additionally, it explores the impact of hyperparameters on model performance through systematic experiments.</p>

**<p style="font-size:18px;">Data Preparation</p>** 
This contains all the data preprocessing and cleaning</p> 

In [829]:
import os
print(os.getcwd())

/Users/imanteh/f20bc/F21BC/Notebooks


In [830]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [831]:
df = pd.read_csv("../Dataset/concrete_data.csv", skiprows=1)

In [832]:
df.head()

,cement,blast_furnace_slag,fly_ash,water,superplasticizer,coarse_aggregate,fine_aggregate,age,concrete_compressive_strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [833]:
# Viewing the dimensionality of the dataset
df.shape

(1030, 9)

In [834]:
df.dropna(inplace = True)
df.isnull().sum()

cement                           0
blast_furnace_slag               0
fly_ash                          0
water                            0
superplasticizer                 0
coarse_aggregate                 0
fine_aggregate                   0
age                              0
concrete_compressive_strength    0
dtype: int64

In [835]:
print(f'Duplicate rows: {df.duplicated().sum()}')

Duplicate rows: 25


In [836]:
# Remove duplicate rows
df.drop_duplicates(inplace=True)

**<p style="font-size:18px;">Scaling and Encoding</p>** 

In [837]:
print(df.head())
print(df.columns)

   cement  blast_furnace_slag  fly_ash  water  superplasticizer  \
0   540.0                 0.0      0.0  162.0               2.5   
1   540.0                 0.0      0.0  162.0               2.5   
2   332.5               142.5      0.0  228.0               0.0   
3   332.5               142.5      0.0  228.0               0.0   
4   198.6               132.4      0.0  192.0               0.0   

   coarse_aggregate  fine_aggregate   age  concrete_compressive_strength  
0            1040.0            676.0   28                          79.99  
1            1055.0            676.0   28                          61.89  
2             932.0            594.0  270                          40.27  
3             932.0            594.0  365                          41.05  
4             978.4            825.5  360                          44.30  
Index(['cement', 'blast_furnace_slag', 'fly_ash', 'water', 'superplasticizer',
       'coarse_aggregate', 'fine_aggregate ', 'age',
       'concret

In [838]:
from sklearn.model_selection import train_test_split

X = df.iloc[:, :-1] # All columns except the last
y = df.iloc[:, -1] # The last column

In [839]:
print(X.shape)
print(y.shape)

(1005, 8)
(1005,)


In [840]:
print(X.head())

   cement  blast_furnace_slag  fly_ash  water  superplasticizer  \
0   540.0                 0.0      0.0  162.0               2.5   
1   540.0                 0.0      0.0  162.0               2.5   
2   332.5               142.5      0.0  228.0               0.0   
3   332.5               142.5      0.0  228.0               0.0   
4   198.6               132.4      0.0  192.0               0.0   

   coarse_aggregate  fine_aggregate   age  
0            1040.0            676.0   28  
1            1055.0            676.0   28  
2             932.0            594.0  270  
3             932.0            594.0  365  
4             978.4            825.5  360  


In [841]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [842]:
print("First 5 rows of scaled training data:")
print(X_train_scaled[:5])

First 5 rows of scaled training data:
[[-0.7036142   0.7362629  -0.8810702   0.1799759  -1.0226942   1.32341753
  -0.17765612 -0.62455318]
 [-0.83112259 -0.84582676  1.04788182 -0.72981642  0.63596651  1.35948653
   0.32102131  0.88513966]
 [ 0.43140366  1.06620495 -0.8810702   0.38479825 -0.18498677 -1.33280675
   0.00745899 -0.28365479]
 [-1.05233032  0.66864226  1.10026551 -0.30588178  0.28412939  0.42298068
  -0.33758549 -0.51092038]
 [-1.17404287 -0.84582676  1.31904441  0.5419875   0.50193332 -1.24005789
   1.18741037 -0.28365479]]


**<p style="font-size:18px;">Artificial Neural Network (ANN)</p>**
Below are the implementations of the ANN to train and predict concrete compressive strength

In [843]:
class activationFunction:
    def logisticFunction(x):
        return 1 / (1 + np.exp(-x))

    def reluFunction(x):
        return np.maximum(0, x)

    def hyperbolicFunction(x):
        return np.tanh(x)

    def leakyReLU(x, alpha=0.01):
        return np.maximum(x, alpha * x)

    def linearFunction(x):  # Identity function for regression output
        return x
    
    # def elu(x, alpha=1.0):
    #     return np.where(x > 0, x, alpha * (np.exp(x) - 1))

class ArtificialNeuralNetwork:
    def __init__(self, layerSize, activationFunction):
        self.layerSize = layerSize
        self.activationFunction = activationFunction
        self.weights = [np.random.randn(layer_sizes[i], layer_sizes[i + 1]) * np.sqrt(2 / layer_sizes[i]) for i in range(len(layer_sizes) - 1)]
        self.biases = [np.random.randn(1, layer_sizes[i + 1]) for i in range(len(layer_sizes) - 1)]

    def forwardPropagation(self, x):
        output = x
        for i in range(len(self.weights)):
            matrix_total = np.dot(output, self.weights[i]) + self.biases[i]
            output = self.activationFunction[i](matrix_total)  
        return output

Layer size is structure to be 8,16,8,1 because 
- 8 input layers
- 16 neurons in first hidden layer
- 8 for the second hidden layer
- 1 for the output layer<p>
This is to check the predicted values generated after forward propagation from the ANN.</p>

In [844]:
layer_sizes = [8, 16, 8, 1]

activation_functions = [
    activationFunction.logisticFunction,
    activationFunction.reluFunction,
    activationFunction.linearFunction,
]
ann = ArtificialNeuralNetwork(layer_sizes, activation_functions)

x_example = np.random.rand(8, 8)  #8 samples, 8 features
output = ann.forwardPropagation(x_example)
print("ANN Output (Sample Predictions):", output)

ANN Output (Sample Predictions): [[ 0.10046464]
 [-0.09149112]
 [-0.25368316]
 [-0.15078599]
 [-0.24842705]
 [-0.30474659]
 [-0.12645748]
 [-0.32745264]]


**<p style="font-size:18px;">Loss Function</p>**
Since the problem domian is regression, MSE is utilised as the loss function

In [845]:
class lossFunction:
    def evaluate(self,y_pred,y_train):
        self.y_pred=y_pred
        self.y_train=y_train
class MeanSquaredError(lossFunction):
    def evaluate(self, y_pred, y_train):
        return np.mean((y_pred - y_train) ** 2)

In [846]:
y_train = y_train.to_numpy().reshape(-1, 1)
y_pred = ann.forwardPropagation(X_train_scaled) # Forward propagating  through the ANN
# checking y_pred and y_train has the correct shape
print("y_pred shape:", y_pred.shape) 
print("y_train shape:", y_train.shape)

# use to calculate the loss function
loss_function=MeanSquaredError()
loss=loss_function.evaluate(y_pred, y_train)
print("Loss:", loss)

y_pred shape: (703, 1)
y_train shape: (703, 1)
Loss: 1482.7793846307336


**<p style="font-size:18px;">Implement PSO Algorithm</p>**

In [847]:
class Particle:
    def __init__(self,vectorSize):
            self.particlePosition=np.random.rand(vectorSize)
            self.particleVelocity=np.random.rand(vectorSize)
            self.bestPosition=np.copy(self.particlePosition)
            self.informants=[]

In [848]:
def particleToAnn(particle, annLayers, activationFunctions):
    neuralNetwork = ArtificialNeuralNetwork(layerSize=annLayers, activationFunction=activationFunctions)
    weightBiasIndexCount = 0
    for i in range(len(annLayers) - 1):
        prevValue = annLayers[i]
        nextValue = annLayers[i + 1]
        weightRange = prevValue * nextValue
        
        weight = particle.particlePosition[weightBiasIndexCount:weightBiasIndexCount + weightRange].reshape((prevValue, nextValue))
        weightBiasIndexCount += weightRange
        
        biases = particle.particlePosition[weightBiasIndexCount:weightBiasIndexCount + nextValue].reshape((1, nextValue))
        weightBiasIndexCount += nextValue
        
        # activation = activationFunctions[i]
        neuralNetwork.weights[i] = weight
        neuralNetwork.biases[i] = biases
    return neuralNetwork

In [849]:
def assessFitness(particle, dataset, annLayers, activationFunctions, loss_function):
    x, y = dataset
    ann = particleToAnn(particle, annLayers, activationFunctions)
    predictions = ann.forwardPropagation(x) 
    loss = loss_function.evaluate(predictions, y.reshape(-1, 1))
    return loss

In [850]:
class ParticleSwarmOptimisation:
    def __init__(self, swarmSize, alpha, beta, delta, omega, jumpSize, informantCount, vectorSize):
        self.swarmSize = swarmSize
        self.alpha = alpha
        self.beta = beta
        self.delta = delta
        self.omega = omega
        self.jumpSize = jumpSize
        self.informantCount = informantCount
        self.vectorSize = vectorSize
        self.global_best = None
        self.global_best_fitness = float('inf')

    def initInformants(self, informantCount, particleArray):
        for p in particleArray:
            potentialInformants = [potInf for potInf in particleArray if potInf != p]
            p.informants = [random.choice(potentialInformants) for _ in range(informantCount)]

    def get_best_informant(self, particle, dataset, annLayers, activationFunctions, loss_function):
        bestInf = None
        bestFitnessInf = float('-inf')
        for i in particle.informants:
            fitness = assessFitness(i, dataset, annLayers, activationFunctions, loss_function)
            if fitness > bestFitnessInf:
                bestFitnessInf = fitness
                bestInf = i
        return bestInf.particlePosition

    def psoOptimisation(self, swarmSize, alpha, beta, gamma, jumpSize, informantCount, vectorSize,
                        dataset, annLayers, activationFunctions, loss_function, max_iterations=100):
        
        particleArray = [Particle(vectorSize) for _ in range(swarmSize)]
        self.initInformants(informantCount, particleArray)

        best = None
        iteration = 0

        while iteration < max_iterations:
            # update global best
            for p in particleArray:
                particleFitness = assessFitness(p, dataset, annLayers, activationFunctions, loss_function)
                if best is None or particleFitness < assessFitness(best, dataset, annLayers, activationFunctions, loss_function):
                    best = p

            for p in particleArray:
                previousBest = p.bestPosition
                informantsBest = self.get_best_informant(p, dataset, annLayers, activationFunctions, loss_function)
                allBest = best.bestPosition

                b = np.random.uniform(0.0, beta)
                c = np.random.uniform(0.0, gamma)
                d = np.random.uniform(0.0, delta)

                updatedVelocity = (
                    alpha * p.particleVelocity +
                    b * (previousBest - p.particlePosition) +
                    c * (informantsBest - p.particlePosition) +
                    d * (allBest - p.particlePosition)
                )

                p.particleVelocity = updatedVelocity
                p.particlePosition += jumpSize * updatedVelocity

            iteration += 1

        return best.particlePosition


<p style="font-size:18px;">Test out PSO Functionality</p>

In [851]:
# Code to test out PSO functionality
X_train_scaled = np.random.rand(100, 8)  # Simulated scaled training features (100 samples, 8 features)
y_train = np.random.rand(100, 1)  # Simulated training labels (100 samples, 1 output)

# define hyperparameters for PSO 
swarmSize = 10
alpha = 0.5  # inertia weight
beta = 0.5   # cognitive parameter
delta = 0.5  # social parameter
jumpSize = 0.5
informantCount = 3
vectorSize = sum([(annLayers[i] * annLayers[i + 1]) + annLayers[i + 1] for i in range(len(annLayers) - 1)])  # Total weights + biases
max_iterations = 50

# initialize the PSO optimiser
pso = ParticleSwarmOptimisation(
    swarmSize = swarmSize,
    alpha = alpha,
    beta = beta,
    delta = delta,
    omega = omega,
    jumpSize = jumpSize,
    informantCount = informantCount,
    vectorSize = vectorSize
)

# run it into the psoOptimisation
best_parameters = pso.psoOptimisation(
    swarmSize = swarmSize,
    alpha = alpha,
    beta = beta,
    gamma = delta,  # Using delta as gamma as per your code
    jumpSize = jumpSize,
    informantCount = informantCount,
    vectorSize = vectorSize,
    
    dataset = (X_train_scaled, y_train),
    annLayers = annLayers,
    activationFunctions = activationFunctions,
    loss_function= MeanSquaredError(),
    max_iterations = max_iterations
)
print("Optimised ANN Parameters:", best_parameters)

Optimised ANN Parameters: [0.60170174 0.65474355 0.09944419 0.43370716 0.80973973 0.4234242
 0.80529996 0.53757763 0.05815643 0.65684594 0.66052799 0.47272844
 0.77012653 0.48983237 0.54726868 0.63873103 0.68384823 0.81049954
 0.38808024 0.23737868 0.64998342 0.63029443 0.80265387 0.80566356
 0.69747291 0.65095672 0.15906389 0.68932229 0.64114412 0.15112518
 0.49159705 0.39185045 0.14876266 0.14046641 0.68610965 0.23769889
 0.78491811 0.84934511 0.8644671  0.2345328  0.65203148 0.85262414
 0.60370043 0.02030405 0.17173757 0.39808731 0.4156856  0.18377015
 0.63593823 0.61704426 0.66324339 0.34667356 0.52289541 0.6256944
 0.36508618 0.21276897 0.66403357 0.15168743 0.20804113 0.8054793
 0.59509237 0.05191308 0.96517298 0.36620648 0.22479874 0.93183923
 0.78024274 0.83390452 0.27690543 0.18922815 0.55063319 0.60915932
 0.62145828 0.50211983 0.39841566 0.53866401 0.41433925 0.35834503
 0.84687278 0.42641011 0.58312657 0.27953772 0.27798252 0.5189895
 0.71551442 0.12459391 0.41360984 0.6653

<p style="font-size:18px;">Get Fitness Value</p>
The final fitness represents the performance of the ANN after its weights and biases have been optimized by the PSO

In [852]:
# create a dummy particle and assign the optimized parameters
optimised_particle = Particle(vectorSize)
optimised_particle.particlePosition = best_parameters

# convert the optimized particle position back to ANN weights and biases
optimised_ann = particleToAnn(
    optimised_particle, 
    annLayers,
    activationFunctions
)

# run through ANN with the optimised parameters
fitness_value = assessFitness(
    optimised_particle,  # pass the optimised particle for fitness evaluation
    dataset = (X_train_scaled, y_train),
    annLayers = annLayers,
    activationFunctions = activationFunctions,
    loss_function = MeanSquaredError()
)
print("Fitness Value:", fitness_value)

Fitness Value: 0.32897681927214445


<p style="font-size:18px;">Training the ANN</p>

In [853]:
y_train_pred = optimized_ann.forwardPropagation(X_train_scaled)  # predictions for training data

# calculate training loss (Mean Squared Error)
loss_function = MeanSquaredError()  
training_loss = loss_function.evaluate(y_train_pred, y_train)  # Evaluate MSE loss on training set

print(f"Training Loss: {training_loss}")

Training Loss: 0.32897681927214445


<p style="font-size:18px;">Testing the model on the test set</p>

In [854]:
print(f"y_test_pred shape: {y_test_pred.shape}")
print(f"y_test shape: {y_test.shape}")

y_test_pred shape: (302, 1)
y_test shape: (302,)


In [855]:
# make predictions on the test set
y_test_pred = optimized_ann.forwardPropagation(X_test_scaled)  #x_test_scaled is the scaled test data

# calculate the loss on the test set
test_loss = loss_function.evaluate(y_test_pred, y_test, y_train)
print(f"Test Loss: {test_loss}")

TypeError: MeanSquaredError.evaluate() takes 3 positional arguments but 4 were given

In [ ]:
epochs = 100  # Set this to the number of epochs you want
for epoch in range(epochs):
    y_train_pred = optimized_ann.forwardPropagation(X_train_scaled)
    
    loss_function = MeanSquaredError()
    training_loss = loss_function.evaluate(y_train_pred, y_train)

    # Print the training loss
    print(f"Epoch {epoch + 1}/{epochs} - Training Loss: {training_loss}")

    # Make predictions on the test set
    y_test_pred = optimized_ann.forwardPropagation(X_test_scaled)
    test_loss = loss_function.evaluate(y_test_pred, y_test)

    # Print the test loss
    print(f"Test Loss: {test_loss}")